In [1]:
import pandas as pd

In [2]:
job_df = pd.read_csv('joblist.csv')
job_df.shape

(254, 7)

In [4]:
# Counting number of times the same job is posted
repost_df = job_df.groupby(by=['title', 'company', 'salary', 'summary']).size().reset_index(name='repost')
repost_df = repost_df.drop(['salary'], axis=1).sort_values(by=['company', 'title'])
repost_df.shape

(17, 4)

In [5]:
# Removing duplicates (repost)
job_df = job_df.sort_values(by='post_date')
job_df = job_df.drop_duplicates(subset=['title', 'company', 'salary', 'rating', 'summary'], keep='first', inplace=False, ignore_index=True)
job_df = job_df.sort_values(by=['company', 'title'])

In [6]:
# Salary cleanup
job_df['disclosed_salary'] = job_df['salary'].apply(lambda x: 0 if 'undisclosed' in x.lower() else 1)
job_df['salary'] = job_df['salary'].apply(lambda x: x.split('a')[0] if 'undisclosed' not in x.lower() else -1)
job_df['salary'] = job_df['salary'].apply(lambda x: x.replace('RM','').replace(',','') if x!=-1 else x)

job_df['min_salary'] = job_df['salary'].apply(lambda x: x.split(' - ')[0] if x!=-1 else 0)
job_df['min_salary'] = pd.to_numeric(job_df['min_salary'])

job_df['max_salary'] = job_df['salary'].apply(lambda x: x.split(' - ')[1] if x!=-1 and '-' in x else(0 if x==-1 else x))
job_df['max_salary'] = pd.to_numeric(job_df['max_salary'])

job_df['avg_salary'] = ((job_df.min_salary + job_df.max_salary)/2).astype(int)

In [7]:
# Rating
job_df['rating'] = job_df['rating'].apply(lambda x: float(x) if x!='None' else -1)
job_df['rated'] = job_df['rating'].apply(lambda x: 1 if x!=-1 else 0)

In [8]:
# Job title length
job_df['title_len'] = job_df['title'].apply(lambda x: len(x))

In [9]:
# Recent Post Date
job_df['post_date'] = job_df['post_date'].apply(lambda x: x.split('days')[0].replace('0+','1')).astype(int)

In [10]:
df = pd.merge(job_df, repost_df, how='left', on=['company', 'title', 'summary'])

In [11]:
df_out = df.drop(['Unnamed: 0'], axis=1)
df_out.columns = df_out.columns.str.capitalize()
df_out.rename(columns = {'Salary':'Salary_monthly', 'Post_date':'Days_since_posted',
                        'Disclosed_salary':'Salary_disclosed',}, inplace = True)

In [12]:
# Save cleaned data to csv
df_out.to_csv('ds_jobs_data_cleaned.csv', index=False)